**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# The Postgres Operator by Zalando

see https://github.com/zalando/postgres-operator

In [1]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [2]:
git clone https://github.com/zalando/postgres-operator.git

Cloning into 'postgres-operator'...
remote: Enumerating objects: 23105, done.        
remote: Counting objects: 100% (224/224), done.        
remote: Compressing objects: 100% (166/166), done.        
remote: Total 23105 (delta 117), reused 108 (delta 46), pack-reused 22881        
Receiving objects: 100% (23105/23105), 8.83 MiB | 15.89 MiB/s, done.
Resolving deltas: 100% (16538/16538), done.


In [3]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [4]:
cd postgres-operator

In [8]:
kubectl delete -f manifests/configmap.yaml  # configuration
kubectl delete -f manifests/operator-service-account-rbac.yaml  # identity and permissions
kubectl delete -f manifests/postgres-operator.yaml  # deployment
kubectl delete -f manifests/api-service.yaml  # operator API to be used by UI

configmap "postgres-operator" deleted
serviceaccount "postgres-operator" deleted
clusterrole.rbac.authorization.k8s.io "postgres-operator" deleted
clusterrolebinding.rbac.authorization.k8s.io "postgres-operator" deleted
clusterrole.rbac.authorization.k8s.io "postgres-pod" deleted
deployment.apps "postgres-operator" deleted
service "postgres-operator" deleted


In [6]:
helm

The Kubernetes package manager

Common actions for Helm:

- helm search:    search for charts
- helm pull:      download a chart to your local directory to view
- helm install:   upload the chart to Kubernetes
- helm list:      list releases of charts

Environment variables:

| Name                               | Description                                                                       |
|------------------------------------|-----------------------------------------------------------------------------------|
| $HELM_CACHE_HOME                   | set an alternative location for storing cached files.                             |
| $HELM_CONFIG_HOME                  | set an alternative location for storing Helm configuration.                       |
| $HELM_DATA_HOME                    | set an alternative location for storing Helm data.                                |
| $HELM_DEBUG                        | indicate whether or not Helm is running in Debug mode                

In [9]:
helm install postgres-operator ./charts/postgres-operator

NAME: postgres-operator
LAST DEPLOYED: Thu Jan 20 10:16:10 2022
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator"


In [13]:
helm ls

NAME             	NAMESPACE	REVISION	UPDATED                                	STATUS  	CHART                  	APP VERSION
postgres-operator	default  	1       	2022-01-20 10:16:10.674505693 +0000 UTC	deployed	postgres-operator-1.7.1	1.7.1      


In [10]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator"


NAME                                 READY   STATUS    RESTARTS   AGE
postgres-operator-594c75b5fc-t74rl   1/1     Running   0          16s


In [11]:
kubectl get pods -A

NAMESPACE              NAME                                         READY   STATUS    RESTARTS       AGE
default                postgres-operator-594c75b5fc-t74rl           1/1     Running   0              26s
default                sleep-56bc8d768-hfcpw                        1/1     Running   1 (17h ago)    18h
kube-system            calico-kube-controllers-58497c65d5-ld8bt     1/1     Running   1 (17h ago)    18h
kube-system            calico-node-j9h54                            1/1     Running   1 (17h ago)    18h
kube-system            coredns-78fcd69978-8hqqx                     1/1     Running   1 (17h ago)    18h
kube-system            etcd-minikube                                1/1     Running   1 (17h ago)    18h
kube-system            kube-apiserver-minikube                      1/1     Running   1 (17h ago)    18h
kube-system            kube-controller-manager-minikube             1/1     Running   1 (17h ago)    18h
kube-system            kube-proxy-k48tj                

In [12]:
kubectl get all

NAME                                     READY   STATUS    RESTARTS      AGE
pod/postgres-operator-594c75b5fc-t74rl   1/1     Running   0             36s
pod/sleep-56bc8d768-hfcpw                1/1     Running   1 (17h ago)   18h

NAME                        TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/kubernetes          ClusterIP   10.96.0.1      <none>        443/TCP    18h
service/postgres-operator   ClusterIP   10.99.144.36   <none>        8080/TCP   36s

NAME                                READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/postgres-operator   1/1     1            1           36s
deployment.apps/sleep               1/1     1            1           18h

NAME                                           DESIRED   CURRENT   READY   AGE
replicaset.apps/postgres-operator-594c75b5fc   1         1         1       36s
replicaset.apps/sleep-56bc8d768                1         1         1       18h

NAME                                                    I

In [16]:
helm install postgres-operator-ui ./charts/postgres-operator-ui

Error: INSTALLATION FAILED: cannot re-use a name that is still in use


: 1

In [24]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator-ui"

NAME                                    READY   STATUS    RESTARTS   AGE
postgres-operator-ui-58644cfcff-kbxzl   1/1     Running   0          115s


In [ ]:
#kubectl port-forward svc/postgres-operator-ui 8081:80

In [22]:
kubectl get all

NAME                                        READY   STATUS    RESTARTS      AGE
pod/postgres-operator-594c75b5fc-t74rl      1/1     Running   0             6m12s
pod/postgres-operator-ui-58644cfcff-kbxzl   1/1     Running   0             78s
pod/sleep-56bc8d768-hfcpw                   1/1     Running   1 (17h ago)   18h

NAME                           TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/kubernetes             ClusterIP   10.96.0.1      <none>        443/TCP    18h
service/postgres-operator      ClusterIP   10.99.144.36   <none>        8080/TCP   6m12s
service/postgres-operator-ui   ClusterIP   10.97.25.92    <none>        80/TCP     78s

NAME                                   READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/postgres-operator      1/1     1            1           6m12s
deployment.apps/postgres-operator-ui   1/1     1            1           78s
deployment.apps/sleep                  1/1     1            1           18h

NAME               

# In a separate terminal

`kubectl port-forward --address $(hostname -i) svc/postgres-operator-ui 8081:80`

In [25]:
kubectl get secret

NAME                                                      TYPE                                  DATA   AGE
default-token-c52hj                                       kubernetes.io/service-account-token   3      18h
postgres-operator-token-86wck                             kubernetes.io/service-account-token   3      17m
postgres-operator-ui-token-fb4bx                          kubernetes.io/service-account-token   3      13m
postgres-pod-token-l2lst                                  kubernetes.io/service-account-token   3      62s
postgres.acid-mypg.credentials.postgresql.acid.zalan.do   Opaque                                2      61s
sh.helm.release.v1.postgres-operator-ui.v1                helm.sh/release.v1                    1      13m
sh.helm.release.v1.postgres-operator.v1                   helm.sh/release.v1                    1      17m
standby.acid-mypg.credentials.postgresql.acid.zalan.do    Opaque                                2      61s


In [26]:
kubectl get svc 

NAME                   TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
acid-mypg              ClusterIP   10.105.204.137   <none>        5432/TCP   103s
acid-mypg-config       ClusterIP   None             <none>        <none>     71s
acid-mypg-repl         ClusterIP   10.97.175.127    <none>        5432/TCP   103s
kubernetes             ClusterIP   10.96.0.1        <none>        443/TCP    18h
postgres-operator      ClusterIP   10.99.144.36     <none>        8080/TCP   18m
postgres-operator-ui   ClusterIP   10.97.25.92      <none>        80/TCP     13m


In [27]:
kubectl get service acid-mypg-repl -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2022-01-20T10:33:02Z"
  labels:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
    team: acid
  name: acid-mypg-repl
  namespace: default
  resourceVersion: "9277"
  uid: a1a65cf7-80f7-4cfb-974d-f932ced6bbd9
spec:
  clusterIP: 10.97.175.127
  clusterIPs:
  - 10.97.175.127
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - name: postgresql
    port: 5432
    protocol: TCP
    targetPort: 5432
  selector:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [28]:
export PGPASSWORD=$(kubectl get secret postgres.acid-mypg.credentials.postgresql.acid.zalan.do -o 'jsonpath={.data.password}' | base64 -d)
export PGSSLMODE=require

export PGHOST=acid-mypg-repl
#$(kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[0].ip}' )
echo $PGHOST

echo use
PGCOMMAND=$(echo PGPASSWORD=$PGPASSWORD PGSSLMODE=$PGSSLMODE  /usr/bin/psql  -h $PGHOST -U postgres)
echo $PGCOMMAND
echo to connect

acid-mypg-repl
use
PGPASSWORD=iGmdblg1oe74ZkMMYCiyCvvqHN7KrrxA06iR0cy9E5Mv12Hejfkc4svr4cJwCPlg PGSSLMODE=require /usr/bin/psql -h acid-mypg-repl -U postgres
to connect


In [31]:
kubectl get svc

NAME                   TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
acid-mypg              ClusterIP   10.105.204.137   <none>        5432/TCP   4m1s
acid-mypg-config       ClusterIP   None             <none>        <none>     3m29s
acid-mypg-repl         ClusterIP   10.97.175.127    <none>        5432/TCP   4m1s
kubernetes             ClusterIP   10.96.0.1        <none>        443/TCP    18h
postgres-operator      ClusterIP   10.99.144.36     <none>        8080/TCP   20m
postgres-operator-ui   ClusterIP   10.97.25.92      <none>        80/TCP     15m


In [30]:
kubectl get service acid-mypg-repl -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2022-01-20T10:33:02Z"
  labels:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
    team: acid
  name: acid-mypg-repl
  namespace: default
  resourceVersion: "9277"
  uid: a1a65cf7-80f7-4cfb-974d-f932ced6bbd9
spec:
  clusterIP: 10.97.175.127
  clusterIPs:
  - 10.97.175.127
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - name: postgresql
    port: 5432
    protocol: TCP
    targetPort: 5432
  selector:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [33]:
kubectl get service acid-mypg-repl -o jsonpath='{.spec.clusterIP}'

10.97.175.127

In [34]:
kubectl get all

NAME                                        READY   STATUS    RESTARTS      AGE
pod/acid-mypg-0                             1/1     Running   0             5m12s
pod/acid-mypg-1                             1/1     Running   0             4m45s
pod/acid-mypg-2                             1/1     Running   0             4m42s
pod/postgres-operator-594c75b5fc-t74rl      1/1     Running   0             22m
pod/postgres-operator-ui-58644cfcff-kbxzl   1/1     Running   0             17m
pod/sleep-56bc8d768-hfcpw                   1/1     Running   1 (17h ago)   18h

NAME                           TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
service/acid-mypg              ClusterIP   10.105.204.137   <none>        5432/TCP   5m13s
service/acid-mypg-config       ClusterIP   None             <none>        <none>     4m41s
service/acid-mypg-repl         ClusterIP   10.97.175.127    <none>        5432/TCP   5m13s
service/kubernetes             ClusterIP   10.96.0.1        <none>     

In [35]:
kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[0].ip}'

In [36]:
kubectl get service acid-mypg-repl -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2022-01-20T10:33:02Z"
  labels:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
    team: acid
  name: acid-mypg-repl
  namespace: default
  resourceVersion: "9277"
  uid: a1a65cf7-80f7-4cfb-974d-f932ced6bbd9
spec:
  clusterIP: 10.97.175.127
  clusterIPs:
  - 10.97.175.127
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - name: postgresql
    port: 5432
    protocol: TCP
    targetPort: 5432
  selector:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [37]:
echo $PGCOMMAND

PGPASSWORD=iGmdblg1oe74ZkMMYCiyCvvqHN7KrrxA06iR0cy9E5Mv12Hejfkc4svr4cJwCPlg PGSSLMODE=require /usr/bin/psql -h acid-mypg-repl -U postgres


In [42]:
kubectl exec -t  acid-mypg-0 -- /bin/bash -c "$PGCOMMAND -c 'select datname FROM pg_database'"

  datname  
-----------
 postgres
 template1
 template0
(3 rows)



In [39]:
kubectl get pods


NAME                                    READY   STATUS    RESTARTS      AGE
acid-mypg-0                             1/1     Running   0             7m37s
acid-mypg-1                             1/1     Running   0             7m10s
acid-mypg-2                             1/1     Running   0             7m7s
postgres-operator-594c75b5fc-t74rl      1/1     Running   0             24m
postgres-operator-ui-58644cfcff-kbxzl   1/1     Running   0             19m
sleep-56bc8d768-hfcpw                   1/1     Running   1 (18h ago)   18h


In [48]:
kubectl logs acid-mypg-1 | grep  --color=always -E '$|leader'

2022-01-20 10:33:32,984 - bootstrapping - INFO - Figuring out my environment (Google? AWS? Openstack? Local?)
2022-01-20 10:33:33,439 - bootstrapping - INFO - Could not connect to 169.254.169.254, assuming local Docker setup
2022-01-20 10:33:33,440 - bootstrapping - INFO - No meta-data available for this provider
2022-01-20 10:33:33,441 - bootstrapping - INFO - Looks like your running local
2022-01-20 10:33:33,483 - bootstrapping - INFO - Configuring pgbouncer
2022-01-20 10:33:33,484 - bootstrapping - INFO - No PGBOUNCER_CONFIGURATION was specified, skipping
2022-01-20 10:33:33,485 - bootstrapping - INFO - Configuring crontab
2022-01-20 10:33:33,486 - bootstrapping - INFO - Skipping creation of renice cron job due to lack of SYS_NICE capability
2022-01-20 10:33:33,488 - bootstrapping - INFO - Configuring pgqd
2022-01-20 10:33:33,490 - bootstrapping - INFO - Configuring wal-e
2022-01-20 10:33:33,540 - bootstrapping - INFO - Configuring log
2022-01-20 10:33:33,541 - bootstrapping - INFO 

In [46]:
kubectl delete pod acid-mypg-0

pod "acid-mypg-0" deleted


In [49]:
kubectl get serviceaccount

NAME                   SECRETS   AGE
default                1         19h
postgres-operator      1         45m
postgres-operator-ui   1         40m
postgres-pod           1         28m


In [50]:
kubectl get rolebinding

NAME           ROLE                       AGE
postgres-pod   ClusterRole/postgres-pod   28m


In [51]:
kubectl get rolebinding postgres-pod -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  creationTimestamp: "2022-01-20T10:33:02Z"
  name: postgres-pod
  namespace: default
  resourceVersion: "9268"
  uid: ccdb49bb-6f98-4512-bc96-7d1da6d7544b
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: postgres-pod
subjects:
- kind: ServiceAccount
  name: postgres-pod


In [52]:
kubectl get clusterrole postgres-pod -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  annotations:
    meta.helm.sh/release-name: postgres-operator
    meta.helm.sh/release-namespace: default
  creationTimestamp: "2022-01-20T10:16:10Z"
  labels:
    app.kubernetes.io/instance: postgres-operator
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: postgres-operator
    helm.sh/chart: postgres-operator-1.7.1
  name: postgres-pod
  resourceVersion: "8478"
  uid: 1c8e04f0-319f-4dc9-b94c-ba48cfee5fa4
rules:
- apiGroups:
  - ""
  resources:
  - endpoints
  verbs:
  - create
  - delete
  - deletecollection
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - pods
  verbs:
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - services
  verbs:
  - create


In [53]:
kubectl delete clusterrole postgres-pod

clusterrole.rbac.authorization.k8s.io "postgres-pod" deleted


In [55]:
kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  creationTimestamp: "2021-12-02T09:27:08Z"
  name: postgres-pod
  namespace: default
  resourceVersion: "47244"
  uid: 411eb7be-b67a-48d3-8df7-c30a6dc603ec
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: postgres-pod
subjects:
- kind: ServiceAccount
  name: postgres-pod
EOF

Error from server (Conflict): error when applying patch:
{"metadata":{"annotations":{"kubectl.kubernetes.io/last-applied-configuration":"{\"apiVersion\":\"rbac.authorization.k8s.io/v1\",\"kind\":\"RoleBinding\",\"metadata\":{\"annotations\":{},\"creationTimestamp\":\"2021-12-02T09:27:08Z\",\"name\":\"postgres-pod\",\"namespace\":\"default\",\"resourceVersion\":\"47244\",\"uid\":\"411eb7be-b67a-48d3-8df7-c30a6dc603ec\"},\"roleRef\":{\"apiGroup\":\"rbac.authorization.k8s.io\",\"kind\":\"Role\",\"name\":\"postgres-pod\"},\"subjects\":[{\"kind\":\"ServiceAccount\",\"name\":\"postgres-pod\"}]}\n"},"creationTimestamp":"2021-12-02T09:27:08Z","resourceVersion":"47244","uid":"411eb7be-b67a-48d3-8df7-c30a6dc603ec"},"roleRef":{"kind":"Role"}}
to:
Resource: "rbac.authorization.k8s.io/v1, Resource=rolebindings", GroupVersionKind: "rbac.authorization.k8s.io/v1, Kind=RoleBinding"
Name: "postgres-pod", Namespace: "default"
for: "STDIN": Operation cannot be fulfilled on rolebindings.rbac.authorization.

: 1

In [56]:
kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: postgres-pod
rules:
- apiGroups:
  - ""
  resources:
  - endpoints
  verbs:
  - create
  - delete
  - deletecollection
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - pods
  verbs:
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - services
  verbs:
  - create
EOF

role.rbac.authorization.k8s.io/postgres-pod created


In [57]:
kubectl get role postgres-pod -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  creationTimestamp: "2022-01-20T11:03:27Z"
  name: postgres-pod
  namespace: default
  resourceVersion: "11194"
  uid: 15b8908b-ef9a-483b-9d5e-b2d1516119e5
rules:
- apiGroups:
  - ""
  resources:
  - endpoints
  verbs:
  - create
  - delete
  - deletecollection
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - pods
  verbs:
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - services
  verbs:
  - create


In [58]:
kubectl auth can-i --as default:postgres-pod create pods

no


: 1

In [59]:
kubectl get clusterrole postgres-operator-ui -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  annotations:
    meta.helm.sh/release-name: postgres-operator-ui
    meta.helm.sh/release-namespace: default
  creationTimestamp: "2022-01-20T10:21:04Z"
  labels:
    app.kubernetes.io/instance: postgres-operator-ui
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: postgres-operator-ui
    helm.sh/chart: postgres-operator-ui-1.7.1
  name: postgres-operator-ui
  resourceVersion: "8721"
  uid: 291aa95e-3429-4f83-9a55-985869b7ce30
rules:
- apiGroups:
  - acid.zalan.do
  resources:
  - postgresqls
  verbs:
  - create
  - delete
  - get
  - list
  - patch
  - update
- apiGroups:
  - ""
  resources:
  - pods
  verbs:
  - get
  - list
  - watch
- apiGroups:
  - ""
  resources:
  - services
  verbs:
  - get
  - list
- apiGroups:
  - apps
  resources:
  - deployments
  - statefulsets
  verbs:
  - get
  - list
- apiGroups:
  - ""
  resources:
  - namespaces
  verbs:
  - get
  - list


In [60]:
kubectl get crd

NAME                                                  CREATED AT
bgpconfigurations.crd.projectcalico.org               2022-01-19T15:48:10Z
bgppeers.crd.projectcalico.org                        2022-01-19T15:48:10Z
blockaffinities.crd.projectcalico.org                 2022-01-19T15:48:10Z
clusterinformations.crd.projectcalico.org             2022-01-19T15:48:10Z
felixconfigurations.crd.projectcalico.org             2022-01-19T15:48:10Z
globalnetworkpolicies.crd.projectcalico.org           2022-01-19T15:48:10Z
globalnetworksets.crd.projectcalico.org               2022-01-19T15:48:11Z
hostendpoints.crd.projectcalico.org                   2022-01-19T15:48:11Z
ipamblocks.crd.projectcalico.org                      2022-01-19T15:48:11Z
ipamconfigs.crd.projectcalico.org                     2022-01-19T15:48:11Z
ipamhandles.crd.projectcalico.org                     2022-01-19T15:48:11Z
ippools.crd.projectcalico.org                         2022-01-19T15:48:11Z
kubecontrollersconfigurations.crd.p

In [61]:
kubectl get crd postgresqls.acid.zalan.do -o yaml

apiVersion: apiextensions.k8s.io/v1
kind: CustomResourceDefinition
metadata:
  creationTimestamp: "2022-01-20T10:07:37Z"
  generation: 1
  name: postgresqls.acid.zalan.do
  resourceVersion: "8082"
  uid: 3c59a1eb-86c8-4974-80d4-b51eecea6009
spec:
  conversion:
    strategy: None
  group: acid.zalan.do
  names:
    categories:
    - all
    kind: postgresql
    listKind: postgresqlList
    plural: postgresqls
    shortNames:
    - pg
    singular: postgresql
  scope: Namespaced
  versions:
  - additionalPrinterColumns:
    - description: Team responsible for Postgres cluster
      jsonPath: .spec.teamId
      name: Team
      type: string
    - description: PostgreSQL version
      jsonPath: .spec.postgresql.version
      name: Version
      type: string
    - description: Number of Pods per Postgres cluster
      jsonPath: .spec.numberOfInstances
      name: Pods
      type: integer
    - description: Size of the bound volume
      jsonPath: .spec.volume.size
      name: Volume
      t

In [62]:
kubectl get postgresqls.acid.zalan.do 

NAME        TEAM   VERSION   PODS   VOLUME   CPU-REQUEST   MEMORY-REQUEST   AGE   STATUS
acid-mypg   acid   14        5      1Gi      100m          100Mi            32m   Running


In [63]:
kubectl get postgresqls.acid.zalan.do acid-mypg -o yaml

apiVersion: acid.zalan.do/v1
kind: postgresql
metadata:
  creationTimestamp: "2022-01-20T10:33:02Z"
  generation: 2
  labels:
    team: acid
  name: acid-mypg
  namespace: default
  resourceVersion: "11059"
  uid: c093873b-05ca-4329-95c2-10f21c5a5e4e
spec:
  allowedSourceRanges: []
  numberOfInstances: 5
  postgresql:
    version: "14"
  resources:
    limits:
      cpu: 500m
      memory: 500Mi
    requests:
      cpu: 100m
      memory: 100Mi
  teamId: acid
  volume:
    size: 1Gi
status:
  PostgresClusterStatus: Running


In [64]:
kubectl create -f - <<EOF
apiVersion: acid.zalan.do/v1
kind: postgresql
metadata:
  name: acid-mypg-yaml
  namespace: default
spec:
  allowedSourceRanges: null
  numberOfInstances: 3
  postgresql:
    version: "14"
  resources:
    limits:
      cpu: 500m
      memory: 500Mi
    requests:
      cpu: 100m
      memory: 100Mi
  teamId: acid
  volume:
    size: 10Gi
EOF

postgresql.acid.zalan.do/acid-mypg-yaml created


In [66]:
kubectl get all

NAME                                        READY   STATUS    RESTARTS      AGE
pod/acid-mypg-0                             1/1     Running   0             19m
pod/acid-mypg-1                             1/1     Running   0             32m
pod/acid-mypg-2                             1/1     Running   0             32m
pod/acid-mypg-3                             1/1     Running   0             5m34s
pod/acid-mypg-4                             1/1     Running   0             5m30s
pod/acid-mypg-yaml-0                        1/1     Running   0             41s
pod/acid-mypg-yaml-1                        1/1     Running   0             37s
pod/acid-mypg-yaml-2                        1/1     Running   0             33s
pod/postgres-operator-594c75b5fc-t74rl      1/1     Running   0             50m
pod/postgres-operator-ui-58644cfcff-kbxzl   1/1     Running   0             45m
pod/sleep-56bc8d768-hfcpw                   1/1     Running   1 (18h ago)   19h

NAME                           TYPE

In [67]:
kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                             STORAGECLASS   REASON   AGE
pvc-338632dc-df03-461d-a890-ad4939435e0e   1Gi        RWO            Delete           Bound    default/pgdata-acid-mypg-2        standard                34m
pvc-3afe2f3a-0040-4d05-9892-a4f4d748e146   1Gi        RWO            Delete           Bound    default/pgdata-acid-mypg-3        standard                7m20s
pvc-5650c54e-cf16-4cd5-8d8d-346f5efcfd60   10Gi       RWO            Delete           Bound    default/pgdata-acid-mypg-yaml-2   standard                2m19s
pvc-6023b244-ba4c-4810-b4db-049a53f97f20   10Gi       RWO            Delete           Bound    default/pgdata-acid-mypg-yaml-1   standard                2m23s
pvc-603db978-9d33-401b-bd37-8633e928185b   1Gi        RWO            Delete           Bound    default/pgdata-acid-mypg-4        standard                7m16s
pvc-68b61044-f848-40ac-96de-b68aac207f58   1Gi    

In [68]:
kubectl get pvc

NAME                      STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
pgdata-acid-mypg-0        Bound    pvc-8f8553ae-8941-4e97-b923-5b09af14d28f   1Gi        RWO            standard       35m
pgdata-acid-mypg-1        Bound    pvc-68b61044-f848-40ac-96de-b68aac207f58   1Gi        RWO            standard       35m
pgdata-acid-mypg-2        Bound    pvc-338632dc-df03-461d-a890-ad4939435e0e   1Gi        RWO            standard       35m
pgdata-acid-mypg-3        Bound    pvc-3afe2f3a-0040-4d05-9892-a4f4d748e146   1Gi        RWO            standard       7m56s
pgdata-acid-mypg-4        Bound    pvc-603db978-9d33-401b-bd37-8633e928185b   1Gi        RWO            standard       7m52s
pgdata-acid-mypg-yaml-0   Bound    pvc-dfadb7d4-645c-4f8f-8412-b4aee6ef6b1a   10Gi       RWO            standard       3m3s
pgdata-acid-mypg-yaml-1   Bound    pvc-6023b244-ba4c-4810-b4db-049a53f97f20   10Gi       RWO            standard       2m59s
pgdata-ac

# Storage in Kubernetes

https://kubernetes.io/docs/concepts/storage/

In [69]:
kubectl get pvc pgdata-acid-mypg-0 -o yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  annotations:
    pv.kubernetes.io/bind-completed: "yes"
    pv.kubernetes.io/bound-by-controller: "yes"
    volume.alpha.kubernetes.io/storage-class: default
    volume.beta.kubernetes.io/storage-provisioner: k8s.io/minikube-hostpath
  creationTimestamp: "2022-01-20T10:33:03Z"
  finalizers:
  - kubernetes.io/pvc-protection
  labels:
    application: spilo
    cluster-name: acid-mypg
  name: pgdata-acid-mypg-0
  namespace: default
  resourceVersion: "9306"
  uid: 8f8553ae-8941-4e97-b923-5b09af14d28f
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: standard
  volumeMode: Filesystem
  volumeName: pvc-8f8553ae-8941-4e97-b923-5b09af14d28f
status:
  accessModes:
  - ReadWriteOnce
  capacity:
    storage: 1Gi
  phase: Bound


In [72]:
ls -lF /tmp/hostpath-provisioner/default

total 32
drwxrwxrwx 3 root root 4096 Jan 20 10:33 pgdata-acid-mypg-0/
drwxrwxrwx 3 root root 4096 Jan 20 10:33 pgdata-acid-mypg-1/
drwxrwxrwx 3 root root 4096 Jan 20 10:33 pgdata-acid-mypg-2/
drwxrwxrwx 3 root root 4096 Jan 20 11:00 pgdata-acid-mypg-3/
drwxrwxrwx 3 root root 4096 Jan 20 11:01 pgdata-acid-mypg-4/
drwxrwxrwx 3 root root 4096 Jan 20 11:05 pgdata-acid-mypg-yaml-0/
drwxrwxrwx 3 root root 4096 Jan 20 11:05 pgdata-acid-mypg-yaml-1/
drwxrwxrwx 3 root root 4096 Jan 20 11:05 pgdata-acid-mypg-yaml-2/


## StorageClass

https://kubernetes.io/docs/concepts/storage/storage-classes/

## ReclaimPolicy

https://kubernetes.io/docs/concepts/storage/persistent-volumes/#persistentvolumeclaims

## Netapp

https://github.com/NetApp/trident